In [1]:
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=False)

import os
import enum
import logging
import shutil, sys                                                                                                                                                    
import numpy as np
import tensorflow as tf
from matplotlib import pyplot
from keras import backend as K
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from keras.datasets import mnist as mn
from keras.preprocessing.image import ImageDataGenerator

K.set_image_dim_ordering('th')



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Using TensorFlow backend.


In [2]:
# Training Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 128
n_images = 5000

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit

# Load the Mnist test data
xTest = mnist.test.images[:n_images]
yTest = list(mnist.test.labels[:n_images])
xTest = xTest.reshape(xTest.shape[0], 1, 28, 28)
# # convert from int to float
xTest = xTest.astype('float32')
xTestBackup = xTest.copy()
yTestBackup = yTest.copy()

xTrain = mnist.train.images[:10000]
yTrain = list(mnist.train.labels[:10000])
xTrain = xTrain.reshape(xTrain.shape[0], 1, 28, 28)
xTrain = xTrain.astype('float32')
xTrainBackup = xTrain.copy()
yTrainBackup = yTrain.copy()

In [3]:
Shade = "Shade"
Rotate = "Rotate"
Sheer = "Sheer"
Shift = "Shift"

Test = "Test"
Train = "Train"

In [4]:
# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 32 filters 
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)

        # Convolution Layer with 64 filters 
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)

    return out

# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [ ]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

# Define the model function (following TF Estimator Template)
def nn_model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [6]:
def reloadData(dataType):
    if dataType == "Test": # Reset test data
        global xTest
        global yTest
        xTest[:] = xTestBackup
        yTest[:] = yTestBackup
    elif dataType == "Train": # Reset training data
        global xTrain
        global yTrain
        xTrain[:] = xTrainBackup
        yTrain[:] = yTrainBackup
        

In [7]:
def TestModel(model, xData, yData, shuffle):
    input_fn = tf.estimator.inputs.numpy_input_fn(
            x={'images': xData}, y= np.array(yData),
            batch_size=batch_size, shuffle=shuffle)
    accuracy = model.evaluate(input_fn)
    # Prepare the input data
    input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'images': xData}, shuffle=shuffle)
    # Use the model to predict the images class
    preds = list(model.predict(input_fn))
    return preds, accuracy['accuracy']

In [8]:
def TrainModel(model, xTrain, yTrain, shuffle):
    input_fn = tf.estimator.inputs.numpy_input_fn(
            x={'images': xTrain}, y= np.array(yTrain),
            batch_size=batch_size, shuffle=shuffle)
    # Train the Model
    model.train(input_fn, steps=num_steps)

In [9]:
def Transform( xData, yData, tType, value ):
    
    n_images = len(xData)
    
    # tType = 1 for shade
    if tType == "Shade":
        Xnew = [[[[v-value if v-value>0.0 else 0.0 for v in n] for n in x[0]]] for x in xData]
        Xnew = np.array(Xnew)
        xData[:] = Xnew.astype('float32')
        return;
        
    # tType = 2 for rotation
    if tType == "Rotate":
        datagen = ImageDataGenerator(rotation_range=value)# fit parameters from data
                
    # tType = 3 for sheer
    if tType == "Sheer":
        datagen = ImageDataGenerator(shear_range=value)# fit parameters from data
        
    # tType = 3 for shift
    if tType == "Shift":   
        datagen = ImageDataGenerator(width_shift_range=value, height_shift_range=value)
        
    datagen.fit(xData)
    # configure batch size and retrieve one batch of images
    for xBatch, yBatch in datagen.flow(xData, yData, batch_size=n_images, shuffle=False):
        xData[:] = xBatch
        yData[:] = yBatch
        break

In [11]:
def DisplayData( xTest, yTest ):
    for i in range(len(xTest)):
        plt.imshow(np.reshape(xTest[i], [28, 28]), cmap='gray')
        plt.show()
        print ("Image Label: ", yTest[i])

In [12]:
def OpenFile(MT):
    filename = str(MT)+".txt"
    fp = open(filename, 'w')
    return fp

def CloseFile(fp):
    fp.close()
    
logging.getLogger("tensorflow").setLevel(logging.INFO)
#Shade = "Shade"
#Rotate = "Rotate"
#Sheer = "Sheer"
#Shift = "Shift"

In [13]:
def ShuffleMT (mode, fp, iteration):
    print ("Reloading training and test data")
    reloadData(Train)
    reloadData(Test)
    
    if (mode==Train):
        print ("Training model")
        TrainModel(xTrain, yTrain, True)
        
        print ("Evaluating Model")
        accuracy = EvaluateModel(xTest, yTest, False)
          
    elif (mode==Test):
        print ("Evaluating Model")
        accuracy = EvaluateModel(xTest, yTest, True)
        
    elif (mode=="Both"):
        print ("Training model")
        TrainModel(xTrain, yTrain, True)       
        print ("Evaluating Model")
        accuracy = EvaluateModel(xTest, yTest, True)

    print("Iteration: ", iteration, " Accuracy: ", accuracy)
    if (fp):
        fp.write(str(iteration)+"\t"+str(accuracy)+"\n")